# V3 Fraction Collector (Runze Valve)

In [1]:
from fraction_collector_v4 import FractionCollector
from sample_producer_flowreactor import SampleProducer
import threading
import queue
import time

In [2]:
# Initialize the FractionCollector
collector = FractionCollector(sensor_id=1, runze_valve_port='COM9', runze_valve_address=0, runze_valve_num_port=10, collection_num=3, waste_num=6)

Connected to CNC Machine!
Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']


In [3]:
# Define well locations
loc = 'well_plate_location'
#loc = '96_well_plate_location'

In [ ]:
# Shared queue and event
sample_queue = queue.Queue()
stop_event = threading.Event()
number_of_samples = 1

# Initialize producer
producer = SampleProducer(
    reaction_profile_csv='HCW_5.csv',
    sample_queue=sample_queue,
    stop_event=stop_event,
    tubing_length_m=1.0,
    tubing_id_inch=0.03,
    buffer_factor=1.2
)
producer.start()

[17:00:58] === Sample Collection Summary ===
[17:00:58] Reaction name         : HCW_3
[17:00:58] Waste time (s)        : 1548.3
[17:00:58] Collection time (s)   : 781.9
[17:00:58] Flow rate (mL/min)    : 0.50
[17:00:58] Tubing length (m)     : 1.0
[17:00:58] Tubing ID (inch)      : 0.03
[17:00:58] Buffer factor         : 1.2
[17:00:58] Tubing delay (s)      : 54.7
[17:00:58] Adjusted waste time   : 1614.0 s
[17:00:58] Adjusted collect time : 651.6 s
[17:00:58] =================================
[17:00:58] Starting waste phase for 1614.0 seconds...


[17:01:28] Wasting... 30 / 1614 seconds elapsed
[17:01:58] Wasting... 60 / 1614 seconds elapsed
[17:02:28] Wasting... 90 / 1614 seconds elapsed
[17:02:58] Wasting... 120 / 1614 seconds elapsed
[17:03:28] Wasting... 150 / 1614 seconds elapsed
[17:03:58] Wasting... 180 / 1614 seconds elapsed
[17:04:28] Wasting... 210 / 1614 seconds elapsed
[17:04:58] Wasting... 240 / 1614 seconds elapsed
[17:05:28] Wasting... 270 / 1614 seconds elapsed
[17:05:58] Wasting... 300 / 1614 seconds elapsed
[17:06:28] Wasting... 330 / 1614 seconds elapsed
[17:06:58] Wasting... 360 / 1614 seconds elapsed
[17:07:28] Wasting... 390 / 1614 seconds elapsed
[17:07:58] Wasting... 420 / 1614 seconds elapsed
[17:08:28] Wasting... 450 / 1614 seconds elapsed
[17:08:58] Wasting... 480 / 1614 seconds elapsed
[17:09:28] Wasting... 510 / 1614 seconds elapsed
[17:09:58] Wasting... 540 / 1614 seconds elapsed
[17:10:28] Wasting... 570 / 1614 seconds elapsed
[17:10:58] Wasting... 600 / 1614 seconds elapsed
[17:11:28] Wasting... 6

In [ ]:
# Simulate collection
loc_index = 0
loc_threshold = loc_index + number_of_samples
collection_thread = None

while not stop_event.is_set():
    try:
        sample = sample_queue.get(timeout=0.1)
        if collection_thread and collection_thread.is_alive():
            print("❌ Not enough time between samples — shutting down.")
            stop_event.set()
            break

        #collection_thread = threading.Thread(target=collect_sample_sim, args=(sample, loc_index, 3))
        collection_thread = threading.Thread(target=collector.collect_fraction, args=(70, loc, loc_index, 20)) #real collection
        collection_thread.start()
        loc_index += 1

    except queue.Empty:
        pass

    if loc_index >= loc_threshold:  # match number of samples
        break

# Wait for last collection to finish
if collection_thread and collection_thread.is_alive():
    collection_thread.join()

# Final result
if stop_event.is_set() or loc_index < loc_threshold:
    print("❌ Not enough samples")
else:
    print("✅ All samples collected successfully.")

Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']
Rinsing collection tubing for 20 drops...

Current port set to 3
Drop Counter (Drops) Started
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Current port set to 6
Rinsing complete.

Moving to location: well_plate_location at index 0
Movement commands rendered: 4
Moved safely To (X35.5, Y129.5, Z-12):  ['ok']

Current port set to 3
Starting fraction collection at well_plate_location (index 0)...

Drop Counter (Drops) Started
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Fraction collection of well_plate_location (index 0) complete.

Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']
✅ All samples collected successfully.


In [5]:
# Simulate collection
loc_index = 0
loc_threshold = loc_index + number_of_samples*3
collection_thread = None

while not stop_event.is_set():
    try:
        sample = sample_queue.get(timeout=0.1)
        if collection_thread and collection_thread.is_alive():
            print("❌ Not enough time between samples — shutting down.")
            stop_event.set()
            break
        for i in range(3): # Collect 3 fractions per sample
            #collection_thread = threading.Thread(target=collect_sample_sim, args=(sample, loc_index, 3))
            collection_thread = threading.Thread(target=collector.collect_fraction, args=(50, loc, loc_index, 20)) #real collection
            collection_thread.start()
            collection_thread.join()
            loc_index += 1

    except queue.Empty:
        pass

    if loc_index >= loc_threshold:  # match number of samples
        break

# Wait for last collection to finish
if collection_thread and collection_thread.is_alive():
    collection_thread.join()

# Final result
if stop_event.is_set() or loc_index < loc_threshold:
    print("❌ Not enough samples")
else:
    print("✅ All samples collected successfully.")

Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']
Rinsing collection tubing for 20 drops...

Current port set to 3
Drop Counter (Drops) Started
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Current port set to 6
Rinsing complete.

Moving to location: well_plate_location at index 0
Movement commands rendered: 4
Moved safely To (X35.5, Y132, Z-12):  ['ok']

Current port set to 3
Starting fraction collection at well_plate_location (index 0)...

Drop Counter (Drops) Started
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
Timeout reached.
Failed to collect enough drops at well_plate_location (index 0)...

Current port set to 6
Current port set to 6
Moving to location: cnc_waste_location at index 0
Movement commands rendered: 4
Moved safely To (X174, Y20, Z0):  ['ok']
Rinsing collection tubing for 20 drops...

Current port set to 

In [5]:
stop_event.set()